In [0]:
## Analyze Query Plans

spark.sql("""
SELECT *
FROM ecommerce.silver.events
WHERE event_type = 'purchase'
""").explain(True)


== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [ecommerce, silver, events], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, ingestion_time: timestamp
Project [event_time#13192, event_type#13193, product_id#13194, category_id#13195L, category_code#13196, brand#13197, price#13198, user_id#13199, user_session#13200, ingestion_time#13201]
+- Filter (event_type#13193 = purchase)
   +- SubqueryAlias ecommerce.silver.events
      +- Relation ecommerce.silver.events[event_time#13192,event_type#13193,product_id#13194,category_id#13195L,category_code#13196,brand#13197,price#13198,user_id#13199,user_session#13200,ingestion_time#13201] parquet

== Optimized Logical Plan ==
Filter (isnotnull(event_type#13193) AND (event_type#13193 = purchase))
+- Relation ecommerce.silver.events[event_ti

In [0]:
%sql
-- Create Partitioned Silver Table

CREATE TABLE ecommerce.silver.events_part
USING DELTA
PARTITIONED BY (event_type)
AS
SELECT
    *,
    DATE(event_time) AS event_date
FROM ecommerce.silver.events;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Apply OPTIMIZE + ZORDER

OPTIMIZE ecommerce.silver.events_part
ZORDER BY (user_id, product_id);



path,metrics
,"List(16, 5, List(35723510, 56036487, 4.421413775E7, 16, 707426204), List(102392726, 200865981, 1.422509962E8, 5, 711254981), 3, List(minCubeSize(107374182400), List(0, 0), List(7, 757968149), 0, List(5, 711254981), 1, null), null, 0, 1, 7, 2, false, 0, 0, 1768625582129, 1768625607482, 8, 1, null, List(0, 0), null, 11, 11, 71370, 0, null)"


In [0]:
%sql
--Check query plan
SELECT * 
FROM ecommerce.silver.events_part
WHERE user_id = 12;


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,ingestion_time,event_date


**Benchmark improvements**

In [0]:
import time
start = time.time()
spark.sql("""
SELECT *
FROM ecommerce.silver.events
WHERE user_id = 12345
""").count()

print(f"Time before optimization: {time.time() - start:.2f} seconds")


Time before optimization: 0.75 seconds


In [0]:
start = time.time()
spark.sql("""
SELECT *
FROM ecommerce.silver.events_part
WHERE user_id = 12345
""").count()

print(f"Time after optimization: {time.time() - start:.2f} seconds")


Time after optimization: 0.49 seconds
